# Study Bayes

### Chapters 8 & 18 
- world cup 

#### Not working:
- empiricaldist
- prior (is part of empiricladist?)
- utils

https://stackoverflow.com/questions/27064348/no-module-named-utils-error-on-compiling-py-file

In [2]:
!pip install empiricaldist

In [1]:
import pandas as pd
import numpy as np
import scipy as scipy
from scipy import linalg, optimize
from scipy.stats import poisson
from scipy.stats import rv_continuous
import matplotlib.pyplot as plt
from empiricaldist import Pmf

In [2]:
df = pd.read_csv('./wc_goals.csv')

In [3]:
df.head()

,year,goals_per_game
0,1930,3.89
1,1934,4.12
2,1938,4.67
3,1950,4.00
4,1954,5.38


In [4]:
lam = 1.4
dist = poisson(lam)
type(dist)
scipy.stats._distn_infrastructure.rv_frozen

scipy.stats._distn_infrastructure.rv_frozen

In [5]:
k = 4
dist.pmf(k)

0.039471954028253146

In [6]:
def make_poisson_pmf(lam, qs):
# """Make a Pmf of a Poisson distribution."""
    ps = poisson(lam).pmf(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

In [7]:
lam = 1.4
goals = np.arange(10)
pmf_goals = make_poisson_pmf(lam, goals)

In [8]:
from scipy.stats import gamma
alpha = 1.4
qs = np.linspace(0, 10, 101)
ps = gamma(alpha).pdf(qs)

In [9]:
prior = Pmf(ps, qs)
prior.normalize()

9.889360237140306

In [10]:
lam = 1.4
k = 4
poisson(lam).pmf(4)

0.039471954028253146

In [11]:
lams = prior.qs
k = 4
likelihood = poisson(lams).pmf(k)

In [12]:
def update_poisson(pmf, data):
# """Update Pmf with a Poisson likelihood."""
    k = data
    lams = pmf.qs
    likelihood = poisson(lams).pmf(k)
    pmf *= likelihood
    pmf.normalize()


In [13]:
france = prior.copy()
update_poisson(france, 4)

In [14]:
croatia = prior.copy()
update_poisson(croatia, 2)

In [15]:
def prob_gt(pmf1, pmf2):
#     """Compute the probability of superiority."""
    total = 0
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            if q1 > q2:
                total += p1 * p2
    return total

In [16]:
prob_gt(france, croatia)

0.7499366290930155

In [17]:
Pmf.prob_gt(france, croatia)

0.7499366290930174

In [18]:
pmf_seq = [make_poisson_pmf(lam, goals)
for lam in prior.qs]

In [19]:
pred_france = make_mixture(france, pmf_seq)

NameError: name 'make_mixture' is not defined

In [ ]:
pred_croatia = make_mixture(croatia, pmf_seq)

In [20]:
win = Pmf.prob_gt(pred_france, pred_croatia)
win

NameError: name 'pred_france' is not defined

In [ ]:
lose = Pmf.prob_lt(pred_france, pred_croatia)
lose

In [ ]:
tie = Pmf.prob_eq(pred_france, pred_croatia)
tie

In [ ]:
win + tie/2

In [ ]:
def expo_pdf(t, lam):
# """Compute the PDF of the exponential distribution."""
    return lam * np.exp(-lam * t)


In [ ]:
lam = 1.4
qs = np.linspace(0, 4, 101)
ps = expo_pdf(qs, lam)
pmf_time = Pmf(ps, qs)
pmf_time.normalize()